모듈

In [ ]:
import torch
from torch import nn
import numpy as np
import gym
import random
from copy import deepcopy
import ctypes

<span style="font-size:14px;font-family:consolas"> 리플레이 메모리 <br> 리스트에다가 데이터셋 [인덱스, 중요도, 이전 상태, 행위, 보상, 끝 여부, 이후 상태] 를 저장하고, <br> 세그먼트 트리로 중요도에 비례하는 뽑힐 확률을 가지게끔 함.(지난 게시물 참고) <br><br> 그리고 이건 정석적인 방식은 아닌데, <br>중요도만 따로 저장하고 있는 ndarray를 만들어서, 리플레이 메모리가 꽉 차서 새 걸 넣으려면 하나를 지워야 할 때,<br> np.argmin으로 가장 중요도가 낮은 놈을 찾아서 지우게끔 해봤음. </span>

In [ ]:
tree_lib=ctypes.cdll.LoadLibrary("C:/Users/love4/Desktop/pr/c/segtree.dll")

class segtree(object):
    def __init__(self, maxlen):
        tree_lib.tree_edit.argtypes=[ctypes.c_void_p, ctypes.c_int, ctypes.c_double]
        tree_lib.tree_get_idx.argtypes=[ctypes.c_void_p, ctypes.c_double]
        tree_lib.total.argtypes=[ctypes.c_void_p]
        tree_lib.tree_get_idx.restype=ctypes.c_int
        tree_lib.new_tree.restype=ctypes.c_void_p
        tree_lib.total.restype=ctypes.c_double
        self.tree=ctypes.c_void_p(tree_lib.new_tree(maxlen))
    def edit_tree(self, idx, val):
        tree_lib.tree_edit(self.tree, idx, ctypes.c_double(val))
    def get_idx(self, goal):
        return tree_lib.tree_get_idx(self.tree, ctypes.c_double(goal))
    def total(self):
        return tree_lib.total(self.tree)

class replaymem:
    def __init__(self, maxlen):
        self.tree=segtree(maxlen)
        self.memory=[None for _ in range(maxlen+1)]
        self.priority_array=np.zeros(maxlen+1, dtype=np.float32)
        self.len=0
        self.maxlen=maxlen

    def push(self, priority:float, i_state:np.ndarray, action:int, reward:float, done:int, f_state:np.ndarray):
        if self.len<self.maxlen:
            self.tree.edit_tree(self.len+1, priority)
            self.memory[self.len+1]=[self.len+1, priority, i_state, action, reward, done, f_state]
        else:
            edit_idx=np.argmin(self.priority_array[1:])+1
            self.tree.edit_tree(edit_idx, priority-self.memory[edit_idx][1])
            self.memory[edit_idx]=[edit_idx, priority, i_state, action, reward, done, f_state]
        self.len=min(self.len+1, self.maxlen)
        

    def edit_prior(self, idx, priority:float):
        self.tree.edit_tree(idx, priority-self.memory[idx][1])
        self.memory[idx][1]=priority
        self.priority_array[idx]=priority

    def sample(self, size):
        total=self.tree.total()
        for __ in range(size):
            yield self.memory[self.tree.get_idx(random.uniform(0, total))]
    
    def total(self):
        return self.tree.total()

<span style="font-size:14px;font-family:consolas"> 유사 Dueling DQN <br> 모델의 성능은 그닥임. 다만 이리저리 바꿔본 결과 연산속도 대비 정확도는 요게 제일 괜찮았음. <br> 기회가 되면 CNN으로도 구현해 볼 예정. </span>

In [ ]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
        )
        self.advantage=nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 4)
        )
        self.value=nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    def forward(self, x):
        x=self.net(x)
        return self.advantage(x)+self.value(x)


<span style="font-size:14px;font-family:consolas"> 중요도가 높은 기억일수록 자주 떠오를텐데, <br>샘플의 weights를 계산하여 그것이 학습 과정에 어느 정도 영향을 미칠지 beta 변수를 이용해 조절할 수 있음.<br> (어제 올린 도태남 나오는 게시물 참고) <br> 각 중요도들에 alpha 라는 변수를 제곱하는 과정도 있는데 이건 구현 못함. <br> 알고리즘은 주석으로 자세히 비유해서 설명해 놓음 <span> 

In [ ]:
class Agent:
    def __init__(self):
        # 하이퍼 파라미터들 정의
        self.epsilon=1 # 랜덤 행동 할 확률
        self.gamma=.97 # 감가율 (빙빙 도는걸 방지)
        self.eps_decay=.999 # epsilon에다가 이걸 곱해서 epsilon을 조금씩 줄임
        self.alpha=1
        self.beta=.4
        self.e=0.001 # 드물겠지만 중요도가 0으로 업데이트 되는 기억이 발생하는걸 방지
        self.alpha_increase=.001
        self.beta_increase=.00001 
        # beta를 조금씩 늘려 나감. 연애 경험이 많아질수록 
        # 연애 기억에서 얻는 타격이 적어질 거잖음?
        
        # 리플레이 메모리
        self.replaybuff=replaymem(50000)
        
        # main_net과 target_net. 이걸 왜 나누는지는 비유할 방법을 생각중
        self.main_net=DQN().to('cuda')
        self.target_net=DQN().to('cuda')

        self.main_net_optimizer=torch.optim.Adam(self.main_net.parameters())
        
    def decide_action(self, state: np.ndarray) -> int:
        # 행위를 정함. epsilon의 확률로 아무 짓이나 함
        if random.random()<self.epsilon:
            return random.randint(0, 3)
        else:
            return int(torch.argmax(self.main_net.forward(self._to_cuda_tensor(state))))
    
    def _to_cuda_tensor(self, x):
        # 튜플/리스트로 묶인 기억들을 행렬로 바꾼 다음 
        # 그래픽카드에 올려주는 함수
        return torch.tensor(x, device='cuda')

    def learn(self):
        if self.replaybuff.len<256:
            return

        # 128개의 기억을 떠올림.
        idxs, priorities, i_states, actions, rewards, dones, f_states=zip(*self.replaybuff.sample(128))
        priorities=self._to_cuda_tensor(priorities)
        rewards=self._to_cuda_tensor(rewards)
        dones=torch.tensor(dones, dtype=torch.float32, device='cuda')
        i_states=torch.stack(tuple(map(self._to_cuda_tensor, i_states)))
        f_states=torch.stack(tuple(map(self._to_cuda_tensor, f_states)))

        # DDQN을 적용한건데 이건 비유할 방법을 아직 못 찾음. 
        # 암튼 궁극적인 목표는 td_targets와 preds가 모든 상황에 대해 같게 만드는거임.
        td_targets=rewards+self.gamma*self.target_net.forward(f_states)[np.arange(128), torch.argmax(self.main_net.forward(f_states), dim=1)]*(1-dones)
        preds=self.main_net.forward(i_states)[np.arange(128), actions]

        # |td_targets-preds|에 비례하게 각 기억들의 중요도를 업데이트 해줌.
        # 내 예상과 크게 다른 일이 있었던 기억이 더 잘 떠올라야 하잖음?
        # weight를 계산하는 과정은 도태남 게시물 참조
        td_errors=td_targets-preds
        weights=(self.replaybuff.len*priorities/self.replaybuff.total())**(-self.beta)
        weights/=torch.max(weights)
        
        for i in range(128):
            self.replaybuff.edit_prior(idxs[i], float(torch.abs(td_errors[i]))**self.alpha+self.e)        
        
        # td_targets 와 preds 의 오차를 이용해 main_net의 신경망 구조 수정
        self.main_net_optimizer.zero_grad()
        lossfn=nn.SmoothL1Loss()
        loss=(weights*lossfn(preds, td_targets)).mean()
        loss.backward()
        self.main_net_optimizer.step()

    def update_target(self):
        #target_net에다가 main_net을 복사
        self.target_net=deepcopy(self.main_net)
        """ self.epsilon*=self.eps_decay """
        self.beta=min(self.beta+self.beta_increase, 0.9)
        """ self.alpha=min(self.alpha+self.alpha_increase, 1) """
        print(self.epsilon)

In [ ]:
agent=Agent()
env=gym.make("LunarLander-v2")

<span style="font-size:14px;font-family:consolas"> 컴퓨터가 노오오동을 함 </span>

In [ ]:
steps=0
for episode in range(500):
    i_state=env.reset()
    total_reward=0
    while True:
        action=agent.decide_action(i_state)
        f_state, reward, done, _=env.step(action)
        total_reward+=reward
        steps+=1
        agent.learn()
        env.render(mode='rgb_array')
        if steps%50==0:
            agent.update_target()
        agent.replaybuff.push(float(np.max(agent.replaybuff.priority_array))+0.1, np.copy(i_state), action, reward, done, np.copy(f_state))
        if done:   
            break
        else:
            i_state=np.copy(f_state)
    """ if episode==100:
        breakpoint() """
    agent.epsilon*=agent.eps_decay

torch.save(agent.target_net.state_dict(), "lunar_target.pth")

In [ ]:
state=env.reset()
for i in range(1000):
    state, reward, done, _=env.step(int(torch.argmax(agent.main_net.forward(torch.tensor(state, device='cuda')))))
    env.render(mode='rgb_array')
    print(reward)
    if done:
        break